In [21]:
from PIL import Image
import numpy as np

def read_image(file_path):
    # Open the image
    img = Image.open(file_path)
    
    # Convert to grayscale for simplicity
    img_gray = img.convert('L')
    
    # Convert to numpy array
    img_array = np.array(img_gray)
    
    return img_array

# Example usage
img_data = read_image('lena.png')

In [22]:
from collections import Counter

def shannon_fano_coding(data):
    # Count the frequency of each symbol
    freq = Counter(data.flatten())
    
    # Sort symbols by frequency in descending order
    sorted_symbols = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    
    def divide_symbols(symbols):
        if len(symbols) <= 1:
            return {symbols[0][0]: '0'}
        
        total_freq = sum(freq for _, freq in symbols)
        half_freq = total_freq / 2
        
        left, right = [], []
        left_freq = 0
        
        for symbol, freq in symbols:
            if left_freq < half_freq:
                left.append((symbol, freq))
                left_freq += freq
            else:
                right.append((symbol, freq))
        
        code = {}
        code.update({symbol: '0' + bits for symbol, bits in divide_symbols(left).items()})
        code.update({symbol: '1' + bits for symbol, bits in divide_symbols(right).items()})
        
        return code
    
    return divide_symbols(sorted_symbols)

# Example usage
shannon_fano_codes = shannon_fano_coding(img_data)

In [23]:
import heapq

class HuffmanNode:
    def __init__(self, symbol, freq):
        self.symbol = symbol
        self.freq = freq
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.freq < other.freq

def huffman_coding(data):
    # Count the frequency of each symbol
    freq = Counter(data.flatten())
    
    # Create a priority queue of HuffmanNodes
    heap = [HuffmanNode(symbol, f) for symbol, f in freq.items()]
    heapq.heapify(heap)
    
    # Build the Huffman tree
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        parent = HuffmanNode(None, left.freq + right.freq)
        parent.left = left
        parent.right = right
        heapq.heappush(heap, parent)
    
    # Generate codes by traversing the tree
    root = heap[0]
    codes = {}
    
    def generate_codes(node, code=''):
        if node.symbol is not None:
            codes[node.symbol] = code
        else:
            generate_codes(node.left, code + '0')
            generate_codes(node.right, code + '1')
    
    generate_codes(root)
    return codes

# Example usage
huffman_codes = huffman_coding(img_data)

In [25]:
import numpy as np
from PIL import Image

def decompress_data(compressed, codes):
    # Create a reverse mapping of codes
    reverse_codes = {code: symbol for symbol, code in codes.items()}
    
    decompressed = []
    current_code = ''
    for bit in compressed:
        current_code += bit
        if current_code in reverse_codes:
            decompressed.append(reverse_codes[current_code])
            current_code = ''
    
    return np.array(decompressed, dtype=np.uint8)

def reconstruct_image(decompressed_data, original_shape):
    # Reshape the decompressed data to the original image shape
    reconstructed = decompressed_data.reshape(original_shape)
    
    # Convert numpy array to PIL Image
    reconstructed_image = Image.fromarray(reconstructed)
    
    return reconstructed_image


In [24]:
def compress_image(data, coding_func):
    codes = coding_func(data)
    compressed = ''.join(codes[symbol] for symbol in data.flatten())
    return compressed, codes

# Compress using Shannon-Fano coding
shannon_fano_compressed, shannon_fano_codes = compress_image(img_data, shannon_fano_coding)

# Compress using Huffman coding
huffman_compressed, huffman_codes = compress_image(img_data, huffman_coding)

# Compare results
original_size = img_data.size * img_data.itemsize * 8  # Size in bits
shannon_fano_size = len(shannon_fano_compressed)
huffman_size = len(huffman_compressed)

print(f"Original size: {original_size} bits")
print(f"Shannon-Fano compressed size: {shannon_fano_size} bits")
print(f"Huffman compressed size: {huffman_size} bits")
print(f"Shannon-Fano compression ratio: {original_size / shannon_fano_size:.2f}")
print(f"Huffman compression ratio: {original_size / huffman_size:.2f}")

Original size: 2097152 bits
Shannon-Fano compressed size: 2230022 bits
Huffman compressed size: 1957574 bits
Shannon-Fano compression ratio: 0.94
Huffman compression ratio: 1.07


In [26]:
shannon_fano_decompressed = decompress_data(shannon_fano_compressed, shannon_fano_codes)
shannon_fano_reconstructed = reconstruct_image(shannon_fano_decompressed, img_data.shape)

# Decompress and reconstruct Huffman compressed image
huffman_decompressed = decompress_data(huffman_compressed, huffman_codes)
huffman_reconstructed = reconstruct_image(huffman_decompressed, img_data.shape)

#displaying the decompressed images

Opening "/tmp/tmpoyq24d0j.PNG" with ImageMagick (color depth=q16)  (image/png)
Opening "/tmp/tmp_v0_djge.PNG" with ImageMagick (color depth=q16)  (image/png)
